## Calculating Portfolio return 
**Objective:** Portfolio creation is necessory to hedge the associated market risk. There are 2 types risk involved while investing in the marlet.
1. Unsystemetic Risk: This risk is also called diversifiable risk. This can be reduced / minimized by portfolio diversiofication. As a thumb rule if someine invest in 25 risk securites diverside in various sector the risk of total investment will be minimized.
2. Systemetic Risk: This risk inharent with the market. The unpredictabl nature of the economy due recession, war or any other situation which are unavoidable and con not be predicted.

Our objective is to calculate best portfolio weights basis customer long term goal to hedge unsystemetic risk.

To calculate the weights we will use daily return of the stock for a period of time. We will calculate the following

1. Sharp Ratio: This is ratio of mean(expected return) and std of daily return. We will subtract market fized return.
- Sharpe Ratio = (Rp - Rf) / σp
2. To annualize the Sharpe Ratio, we multiply it by the square root of the number of periods in a year.
- ASR = Sharpe Ratio * sqrt(252)

A higher ASR indicates:

- Better risk-adjusted performance
- Higher excess returns per unit of risk

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import matplotlib.pyplot as plt
from utility import *
from scipy.optimize import minimize
import json

## FIXED PARAMS

In [2]:
RISK_FREE_RATE = 0.0675 #as per https://www.grantthornton.in/globalassets/1.-member-firms/india/assets/pdfs/valuationinsights_newequitypremium.pdf 


In [3]:
nifty = pd.read_csv(r'./Data/nifty50 list.csv')
nifty.head()

,Company Name,Industry,Symbol,Series,ISIN Code
0,Adani Enterprises Ltd.,Metals & Mining,ADANIENT,EQ,INE423A01024
1,Adani Ports and Special Economic Zone Ltd.,Services,ADANIPORTS,EQ,INE742F01042
2,Apollo Hospitals Enterprise Ltd.,Healthcare,APOLLOHOSP,EQ,INE437A01024
3,Asian Paints Ltd.,Consumer Durables,ASIANPAINT,EQ,INE021A01026
4,Axis Bank Ltd.,Financial Services,AXISBANK,EQ,INE238A01034


In [4]:
#industrywise company seggrigation
industry_comp = {}
industry_lst = nifty['Industry'].unique().tolist()
for i in industry_lst:
    industry_comp[i] = nifty[nifty['Industry'] == i]['Company Name'].to_list()


In [5]:
save_dict_json(filename="industry_comp",ticker_stock=industry_comp)

File saved


## Checking industrywise best stocks

In [33]:
# Assuming get_ticker_name(), get_stock_data(), calculate_roi(), and calculate_cv() are predefined functions

tickers = get_ticker_name()

# Create empty lists for better performance instead of using DataFrame in the loop
best_roi_stocks_list = []
low_risk_stocks_list = []

for industry in industry_lst:
    stocks = industry_comp.get(industry)

    stock_data_list = []

    # Collect ROI and risk data for all stocks in the industry
    for stock in stocks:
        ticker = tickers.get(stock)
        stock_data = get_stock_data(ticker)
        roi = calculate_roi(stock_data)
        cv = calculate_cv(stock_data)  # Coefficient of variation as a measure of risk
        stock_data_list.append([industry, stock, roi, cv])
    
    # Convert list to DataFrame outside of the loop for efficiency
    df = pd.DataFrame(stock_data_list, columns=['industry', 'Stock', 'roi', 'risk'])

    # Find stock with best ROI
    best_roi_stock = df.loc[df['roi'].idxmax()]
    best_roi_stocks_list.append(best_roi_stock)

    # Find stock with lowest risk (highest risk removed by sorting in ascending order)
    low_risk_stock = df.loc[df['risk'].idxmin()]
    low_risk_stocks_list.append(low_risk_stock)

# Convert lists to DataFrames after the loop
best_roi_stocks = pd.DataFrame(best_roi_stocks_list)
low_risk_stocks = pd.DataFrame(low_risk_stocks_list)

# Sort results
best_roi_stocks.sort_values(by='roi', ascending=False, inplace=True)
low_risk_stocks.sort_values(by='risk', ascending=True, inplace=True)

In [34]:
best_roi_stocks

,industry,Stock,roi,risk
0,Metals & Mining,Adani Enterprises Ltd.,9.940217,13.359482
5,Automobile and Auto Components,Tata Motors Ltd.,3.951465,16.546998
4,Healthcare,Sun Pharmaceutical Industries Ltd.,3.423863,12.160055
0,Telecommunication,Bharti Airtel Ltd.,3.131017,14.051395
0,Power,NTPC Ltd.,2.972516,14.315864
4,Fast Moving Consumer Goods,Tata Consumer Products Ltd.,2.697678,14.963354
1,Oil Gas & Consumable Fuels,Coal India Ltd.,2.363142,17.316922
0,Construction Materials,Grasim Industries Ltd.,2.325265,16.801629
2,Information Technology,LTIMindtree Ltd.,2.320944,18.044857
10,Financial Services,State Bank of India,1.512143,21.837493


In [35]:
low_risk_stocks

,industry,Stock,roi,risk
4,Healthcare,Sun Pharmaceutical Industries Ltd.,3.423863,12.160055
0,Automobile and Auto Components,Bajaj Auto Ltd.,3.100914,13.093600
0,Metals & Mining,Adani Enterprises Ltd.,9.940217,13.359482
0,Telecommunication,Bharti Airtel Ltd.,3.131017,14.051395
0,Power,NTPC Ltd.,2.972516,14.315864
4,Fast Moving Consumer Goods,Tata Consumer Products Ltd.,2.697678,14.963354
0,Information Technology,HCL Technologies Ltd.,2.280572,15.630558
0,Construction Materials,Grasim Industries Ltd.,2.325265,16.801629
1,Oil Gas & Consumable Fuels,Coal India Ltd.,2.363142,17.316922
1,Consumer Durables,Titan Company Ltd.,1.417270,20.833075


In [25]:
# basis above we will choose first 3 stocks basis best return  and first 3 stock basis low risk

best_return_stocks = best_roi_stocks.head(3)['Stock'].to_list()
best_risk_stocks =low_risk_stocks.head(3)['Stock'].to_list()

portfolio = list(set(best_return_stocks+best_risk_stocks))

portfolio

['Bharti Airtel Ltd.',
 'Sun Pharmaceutical Industries Ltd.',
 'Adani Enterprises Ltd.',
 'UltraTech Cement Ltd.',
 'Power Grid Corporation of India Ltd.',
 'Tata Motors Ltd.']

## calculating expected return, Sharp Ratio

In [26]:
#generating consolidted dataframe for portfolio

portfolio_df = consolidated_df(portfolio,tickers)
portfolio_df.head()

,Bharti Airtel Ltd.,Sun Pharmaceutical Industries Ltd.,Adani Enterprises Ltd.,UltraTech Cement Ltd.,Power Grid Corporation of India Ltd.,Tata Motors Ltd.
Date,,,,,,
2019-10-04,-0.015781,-0.005425,NaN,-0.041168,-0.012890,-0.022724
2019-10-07,0.006020,-0.016059,-0.030459,-0.027811,-0.000254,-0.018132
2019-10-09,0.050942,0.010604,0.023291,0.043765,0.006088,0.028525
2019-10-10,0.048472,0.008042,-0.011578,-0.016771,0.007056,-0.034501
2019-10-11,0.014866,-0.001940,-0.003280,0.025059,0.009498,0.037804


In [27]:
#annualized cov matrix
cov_matrix = portfolio_df.cov()*252
cov_matrix

,Bharti Airtel Ltd.,Sun Pharmaceutical Industries Ltd.,Adani Enterprises Ltd.,UltraTech Cement Ltd.,Power Grid Corporation of India Ltd.,Tata Motors Ltd.
Bharti Airtel Ltd.,0.086652,0.022632,0.041996,0.028668,0.022999,0.040162
Sun Pharmaceutical Industries Ltd.,0.022632,0.066413,0.032532,0.022739,0.017308,0.034301
Adani Enterprises Ltd.,0.041996,0.032532,0.289758,0.051624,0.038852,0.085208
UltraTech Cement Ltd.,0.028668,0.022739,0.051624,0.077154,0.023099,0.050407
Power Grid Corporation of India Ltd.,0.022999,0.017308,0.038852,0.023099,0.082126,0.039180
Tata Motors Ltd.,0.040162,0.034301,0.085208,0.050407,0.039180,0.193758


In [28]:
constraints = {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1}
bounds = [(0, 0.4) for _ in range(len(portfolio))]

initial_weights = np.array([1/len(portfolio)]*len(portfolio))
initial_weights



array([0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667,
       0.16666667])

In [29]:
# optimized_results = minimize(sharp_ratio,initial_weights,
#                              args=(portfolio_df,cov_matrix,RISK_FREE_RATE))

optimal_solution = minimize(negative_sharp_ratio, initial_weights, 
                            args=(portfolio_df, cov_matrix, RISK_FREE_RATE),
                            method='SLSQP', bounds=bounds, constraints=constraints)

In [30]:
optimal_weights = optimal_solution.x

In [31]:
print("Optimal Weights:")
for ticker, weight in zip(portfolio, optimal_weights):
    print(f"{ticker}: {weight:.4f}")

Optimal Weights:
Bharti Airtel Ltd.: 0.2082
Sun Pharmaceutical Industries Ltd.: 0.3871
Adani Enterprises Ltd.: 0.1923
UltraTech Cement Ltd.: 0.0000
Power Grid Corporation of India Ltd.: 0.1603
Tata Motors Ltd.: 0.0521


In [32]:
port_expected_return = expected_return(optimal_weights,portfolio_df)
port_Volatility = portfolio_std(optimal_weights,cov_matrix)
port_sharp_ratio = sharp_ratio(optimal_weights,portfolio_df,cov_matrix,RISK_FREE_RATE)

print(f"Expected return on the portfolio: {port_expected_return*100:.3f}%")
print(f"Expected risk on the portfolio: {port_Volatility*100:.3f}%")
print(f"Expected sharp ratio on the portfolio: {port_sharp_ratio*100:.3f}")

Expected return on the portfolio: 39.101%
Expected risk on the portfolio: 22.272%
Expected sharp ratio on the portfolio: 145.255
